## Problem Set 3

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

### 1.1

In [2]:
df = pd.read_csv("/Users/zhengyaojin/Desktop/weibo_data.csv")
df.head()

,location,show_id,episode_num,censor_dummy,log_rating,log_tweet,av_tweets,day_id,mainland_dummy
0,Mainland China,1,1,1,0.475764,0.000000,3.692308,33,1
1,Mainland China,1,2,0,0.468479,0.000000,3.692308,34,1
2,Mainland China,1,3,0,0.581327,1.386294,3.692308,35,1
3,Mainland China,1,4,0,0.547851,0.000000,3.692308,36,1
4,Mainland China,1,5,0,0.483728,1.386294,3.692308,37,1


In [3]:
model = smf.ols(formula = 'log_rating ~ log_tweet', data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             log_rating   R-squared:                       0.111
Model:                            OLS   Adj. R-squared:                  0.111
Method:                 Least Squares   F-statistic:                     987.9
Date:                Thu, 29 Feb 2024   Prob (F-statistic):          2.00e-204
Time:                        17:04:24   Log-Likelihood:                -87.734
No. Observations:                7899   AIC:                             179.5
Df Residuals:                    7897   BIC:                             193.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2664      0.003     81.566      0.0

This regression does not give me the causal effect of tweets on show viewership. The estimate will be biased upwards because shows with high quality will naturally have higher viewership and hence higher number of tweets, which will lead to overestimation of the true effect (biased upward). 

### 1.2

#### a.


In [5]:
mainland_data = df[df.mainland_dummy == 1]
mainland_model = smf.ols(formula = 'log_rating ~ censor_dummy + C(show_id)',data = mainland_data).fit()
print(mainland_model.summary())


                            OLS Regression Results                            
Dep. Variable:             log_rating   R-squared:                       0.881
Model:                            OLS   Adj. R-squared:                  0.878
Method:                 Least Squares   F-statistic:                     294.5
Date:                Thu, 29 Feb 2024   Prob (F-statistic):               0.00
Time:                        17:24:49   Log-Likelihood:                 7841.0
No. Observations:                7899   AIC:                        -1.529e+04
Df Residuals:                    7705   BIC:                        -1.394e+04
Df Model:                         193                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.5577      0.02

The coefficient on the censorship dummy is -0.0122 and is statistically significant. This means that the log rating is 0.0122 lower during the censorship time than normal time. This is in my expectation because if the government censored the Sino Weibo, people would be unable to see positive tweets promoting certain shows and thus would not have the opportunity to alter their perspective, which they might otherwise do if Weibo was not blocked. Hence, the censorship would led to decrease in rating. 

#### b.

In [6]:
mainland_model_2 = smf.ols(formula = 'log_rating ~ censor_dummy',data = mainland_data).fit()
print(mainland_model_2.summary())

                            OLS Regression Results                            
Dep. Variable:             log_rating   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     6.093
Date:                Thu, 29 Feb 2024   Prob (F-statistic):             0.0136
Time:                        17:46:17   Log-Likelihood:                -550.22
No. Observations:                7899   AIC:                             1104.
Df Residuals:                    7897   BIC:                             1118.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.3196      0.003    105.685   

It was necessary to control for show fixed effects in the regression. Without controlling for it, the coefficient on the censor_dummy would suggest that the log rating actually improved during censorship time, as the log rating is 0.0286 higher during the censorship time than normal time. This is because if we do not control for show fixed effects, we would not be able to distinguish variations among various shows, such as the time they broadcast, making it impossible to isolate solely the influence of censorship on log rating. 

#### c.

In [7]:
hongkong_data = df[df.mainland_dummy == 0]
hongkong_model = smf.ols(formula = 'log_rating ~ censor_dummy + C(show_id)',data = hongkong_data).fit()
print(hongkong_model.summary())

                            OLS Regression Results                            
Dep. Variable:             log_rating   R-squared:                       0.974
Model:                            OLS   Adj. R-squared:                  0.973
Method:                 Least Squares   F-statistic:                     967.6
Date:                Thu, 29 Feb 2024   Prob (F-statistic):               0.00
Time:                        17:58:49   Log-Likelihood:                 1799.1
No. Observations:                3528   AIC:                            -3332.
Df Residuals:                    3395   BIC:                            -2512.
Df Model:                         132                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.1792      0.02

The coefficient on the censorship dummy is not statistically significant since the p-value for the coefficient is too large(0.340), which suggests that Hongkong is not affected by the censorship. This aligns with my expectation because people from Hongkong use Twitter rather than Weibo, and therefore it is reasonable that they are not affected by the censorship from mainland China. 

#### d.


In [8]:
did_model = smf.ols(formula = 'log_rating ~ + C(show_id) + censor_dummy + censor_dummy:mainland_dummy',data = df).fit()
print(did_model.summary())

                            OLS Regression Results                            
Dep. Variable:             log_rating   R-squared:                       0.964
Model:                            OLS   Adj. R-squared:                  0.963
Method:                 Least Squares   F-statistic:                     910.8
Date:                Thu, 29 Feb 2024   Prob (F-statistic):               0.00
Time:                        18:17:28   Log-Likelihood:                 9018.9
No. Observations:               11427   AIC:                        -1.738e+04
Df Residuals:                   11100   BIC:                        -1.498e+04
Df Model:                         326                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

Interpretation of relevant coefficients:

Intercept: This is the baseline. This coefficient represents the expected value of log_rating when all predictor variables are 0. This would be the baseline show's expected log_rating value (show 1) when censor_dummy is 0 and the interaction of censor_dummy and mainland_dummy is 0.

C(Show_id): These coefficients represent the difference in log_rating between the baseline (show 1) and each other shows. For example, the coefficient for show 2 at -0.0066 means that show 2's log_rating is expected to be 0.0066 units lower than the baseline show's rating, assuming other variables constant.

censor_dummy: This coefficient shows the effect of censor_dummy on log_rating. A value of 0.0106 suggests that shows that are censored have their log_rating increased by 0.0106 compared to uncensored shows, assuming mainland_dummy is 0. However, this coefficent is not statistically significant with a large p-value. 

censor_dummy:mainland_dummy : This coefficient illustrates the extent to which the log_rating of the mainland group has changed following the treatment, relative to what would have occurred in the Hong Kong group, which were unaffected by the censorship. This coefficient is statistically significant, indicating that the treatment influenced the rating.

### 1.3

#### a.

In [11]:
df["less_5_dummy"] = df.av_tweets < 5
df["less_100_dummy"] = (df.av_tweets >= 5) & (df.av_tweets < 100)
df["more_100_dummy"] = (df.av_tweets >= 100)
df.head()

,location,show_id,episode_num,censor_dummy,log_rating,log_tweet,av_tweets,day_id,mainland_dummy,less_5_dummy,less_100_dummy,more_100_dummy
0,Mainland China,1,1,1,0.475764,0.000000,3.692308,33,1,True,False,False
1,Mainland China,1,2,0,0.468479,0.000000,3.692308,34,1,True,False,False
2,Mainland China,1,3,0,0.581327,1.386294,3.692308,35,1,True,False,False
3,Mainland China,1,4,0,0.547851,0.000000,3.692308,36,1,True,False,False
4,Mainland China,1,5,0,0.483728,1.386294,3.692308,37,1,True,False,False


In [13]:
# less than 5 tweets
less_5_data = df[(df.mainland_dummy == 1) & (df.less_5_dummy == 1)]
less_5_model = smf.ols(formula = 'log_rating ~ censor_dummy + C(show_id)', data = less_5_data).fit()
print(less_5_model.summary())

                            OLS Regression Results                            
Dep. Variable:             log_rating   R-squared:                       0.844
Model:                            OLS   Adj. R-squared:                  0.840
Method:                 Least Squares   F-statistic:                     203.3
Date:                Thu, 29 Feb 2024   Prob (F-statistic):               0.00
Time:                        18:50:54   Log-Likelihood:                 4134.6
No. Observations:                3405   AIC:                            -8091.
Df Residuals:                    3316   BIC:                            -7545.
Df Model:                          88                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.5573      0.01

In [14]:
# at least 5 but less than 100 tweets
less_100_data = df[(df.mainland_dummy == 1) & (df.less_100_dummy == 1)]
less_100_model = smf.ols(formula = 'log_rating ~ censor_dummy + C(show_id)', data = less_100_data).fit()
print(less_100_model.summary())

                            OLS Regression Results                            
Dep. Variable:             log_rating   R-squared:                       0.888
Model:                            OLS   Adj. R-squared:                  0.886
Method:                 Least Squares   F-statistic:                     363.1
Date:                Thu, 29 Feb 2024   Prob (F-statistic):               0.00
Time:                        18:53:27   Log-Likelihood:                 2922.7
No. Observations:                2945   AIC:                            -5717.
Df Residuals:                    2881   BIC:                            -5334.
Df Model:                          63                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.8858      0.01

In [15]:
# at least 100 tweets
more_100_data = df[(df.mainland_dummy == 1) & (df.more_100_dummy == 1)]
more_100_model = smf.ols(formula = 'log_rating ~ censor_dummy + C(show_id)', data = more_100_data).fit()
print(more_100_model.summary())

                            OLS Regression Results                            
Dep. Variable:             log_rating   R-squared:                       0.850
Model:                            OLS   Adj. R-squared:                  0.846
Method:                 Least Squares   F-statistic:                     203.8
Date:                Thu, 29 Feb 2024   Prob (F-statistic):               0.00
Time:                        18:55:09   Log-Likelihood:                 1090.7
No. Observations:                1549   AIC:                            -2095.
Df Residuals:                    1506   BIC:                            -1866.
Df Model:                          42                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.7549      0.01

It appears that shows with less than 5 tweets and shows with at least 5 tweets but less than 100 tweets were affected less by the censorship than shows with large amount of tweets (more than 100 tweets). This might due to the fact that shows with relatively low tweets have relatively less social media exposure than shows that have a lot of tweets per episode. Hence, they were affected less by the censorship and would experience less reduction in rating compared to shows that have many tweets. 

#### b.

In [18]:
did_model_2 = smf.ols(formula = 'log_rating ~ censor_dummy + C(show_id) + censor_dummy:less_100_dummy + censor_dummy:more_100_dummy', data = df[(df.mainland_dummy==1)]).fit()
print(did_model_2.summary())

                            OLS Regression Results                            
Dep. Variable:             log_rating   R-squared:                       0.881
Model:                            OLS   Adj. R-squared:                  0.878
Method:                 Least Squares   F-statistic:                     291.7
Date:                Thu, 29 Feb 2024   Prob (F-statistic):               0.00
Time:                        19:11:42   Log-Likelihood:                 7845.2
No. Observations:                7899   AIC:                        -1.530e+04
Df Residuals:                    7703   BIC:                        -1.393e+04
Df Model:                         195                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In

Interpretation of the coefficients: 

Intercept: This is the expected value of log_rating when all other variables are set to 0. It's the baseline rating for the reference category of show_id, which is show 1, assuming censor_dummy, less_100_dummy, and more_100_dummy are all 0.

C(show_id):These coefficients for show_id represent the difference in the expected log rating between the baseline category. and the specific category x of show_id. For example, show 2 at -0.0064 indicates that show 2's log rating is expected to be 0.0064 units lower than the baseline show's log rating, holding all else constant.

censor_dummy: This coefficient indicates the variation in the predicted log rating between censored shows and uncensored shows, while controlling for other vairables. The negative sign indicates that censorship correlates with a decrease in the log rating, although the impact is very samll. 

censor_dummy:less_100_dummy :This interaction term indicates how the relationship between being censored and log rating changes when less_100_dummy is true compared to when it's false. The positive sign indicates that the negative impact of censorship on the rating is slightly lessened for shows with at least 5 twees and less than 100 tweets equal to 1. However, the effect is very small and not statistically significant (p = 0.778).

censor_dummy:more_100_dummy : This interaction term represents the additional impact on the log rating for shows that are censored and where more_100_dummy equals 1 (have more than 100 tweets). The negative coefficient suggests that for shows that are censored, having more than 100 tweets further reduces the log rating compared to shows that have less than 100 tweets. This effect is statistically significant since its p-value is 0.013.



#### c.

Both of these regressions appear to be quite effective. We are examining the impact of censorship by contrasting one group of shows or a specific location where censorship was enforced with another group of shows or a location that remained unaffected by such censorship. Specifically, we use Hong Kong as a control location, as the censorship regulations did not apply. Similarly, we use a set of shows categorized by lower or moderate amount of tweets as our control group of shows; these were not impacted by the censorship event. By comparing these two sets—affected vs. unaffected—we can isolate and understand the effects of censorship and observe how it influences the ratings or other relevant metrics of the shows within the affected location or category, in contrast to those that remained uncensored. Moreover, the magnitude of coefficients appear to be quite similar in both cases, which further indicates that both regression appear to be very informative. 